## Instrucciones

Edita, elabora y ejecuta los comandos y responde a las preguntas en este mismo documento. Pero **¡atención!** debes descargarlo (en formato `.ipynb`) para poder entregarlo a través del aula virtual. Y si dejas que la sesión expire, perderás el trabajo hecho.

Antes de nada, ejecuta el bloque siguiente para que acaben de intalarse los programas que necesitas.

In [ ]:
system2(command = './preparar_ambiente.sh', wait = TRUE)

## Problema 1
Se ha secuenciado un genoma que parece codificar el fragmento siguiente:

`IGCGIQPHASTAEKKGVRQYRSDDDIVANGEEENAAVLDHIESKQTWSYRRFLNGEVQRN
AYRLIPHNRGVEYFVRTTQVRGNRDSEDLKTPPYTVLIVGFSAWTVIHSKQAREGDAIAS
YLFSRPQQQFNITMNPGATSTSSINSLELADKPDTVYNTVARKALTRMKRLSSQTPKLLG
VTRDEDQFDKLWLPFAQQDKAYEISTLLQAMDYVDSNARNGPDVLYGQAIVIDVSGVDLS
IEMSSFRVLTQSKVDYLSFAELDAHNTCPAERFFAEVYRNLNAQQSVAKSEVKTSRSATQ
IKQKVASMQQEQLTQLSKSRQQVIPPLIEGTEIERV`

1. Crea un archivo en formato fasta con esta secuencia.
2. Corrige los comandos del bloque 1 para que ejecuten un *PSIBLAST* sobre la base de datos Swissprot utilizando como *query* el archivo fasta que has creado. (Si no hubieras conseguido crear el archivo fasta con éxito, usa el archivo "Proteina2.fasta" como *query*).
3. ¿Crees que la proteína podría proceder de un hongo? Indica qué te lleva a pensarlo.

In [ ]:
## Bloque 1. Intento de PSIBLAST.

system2(command = 'pepsiblast',
        args = c('-db', 'Swissprot',
                 '-query', 'unknown.fa',
                 '-out', 'psi.txt',
                 '-outfmt', '"7 qacc sacc pident qstart qend sstart send evalue"',
                 '-evalue', '0.001',
                 '-inclusion_ethresh', '0.001',
                 '-num_iterations', '0'),
       stdout = TRUE,
       wait = TRUE)

## Problema 2

El archivo `homolegs.fasta` contiene las secuencias homólogas a la proteïna anterior encontradas mediante *PSIBLAST*. No están alineadas. Corrige los comandos siguientes para alinear las secuencias de ese archivo y contesta las preguntas siguientes.

In [ ]:
## Bloque 2. Alineamiento

suppressMessages(library('DECIPHER'))
Secuencias <- readDNAStringSet('homolegs.fasta')
Alineadas  <- AlignSeqs(Secuencias)
BrowseSeqs(Alineadas, htmlFile = 'Alineamiento.html', openURL = FALSE)

1. ¿Qué clase de objeto es `Secuencias`, en el bloque anterior?
2. ¿Qué opinión te merece el alineamiento, y por qué?
3. Por defecto, `AlignSeqs()` usa una penalización por apertura de *gaps* de entre -18 y -16, y una penalización por alargamiento del *gap* de entre -2 y -1. Comprueba y describe el efecto de invertir esos valores. Es decir, el efecto de asignar el valor -2 (por ejemplo) a la penalización por apertura de *gaps* y el valor de -16 a la penalización por elongamiento del gap.

## Problema 3

El archivo `results.txt` contiene los resultados de una consulta realizada con el paquete `hmmer`. Concretamente, se ha comprobado qué dominios de la base de datos Pfam se encuentran presente en cada una de las proteínas del *Mycobacterium virus Che8*. El código siguiente te permite cargar la tabla de resultados en la sesión de R. Aunque para contestar algunas de las preguntas siguientes basta con abrir el archivo `results.txt` con el editor de texto, o con el comando `less -S results.txt` desde la terminal Bash.

In [ ]:
## Bloque 3. Resultados de hmmer

library(rhmmer)
resultados <- read_domtblout('results.txt')
resultados
names(resultados)

1. El paquete `hmmer` incluye varios programas: *hmmbuild*, *hmmsearch*, *hmmfetch*...  ¿Cuál de esos programas crees que ha sido utilizado para generar los resultados guardados en el archivo `results.txt`?
2. A simple vista, parece que las dos primeras líneas de la tabla de resultados son idénticas. ¿Por qué se muestra dos veces que el dominio *Phage_integrase* està presente en la proteína Q855G4?
3. El número de proteínas en las que se han buscado dominios conocidos es 112. Usando el comando `length(unique(resultados$query_name))` vemos que sólo se han encontrado dominios proteicos conocidos en 57 de esas proteínas. ¿Sabrías decir cuántos dominios diferentes se han encontrado en el proteoma del *Mycobacterium virus Che8*?
4. Los resultados de la búsqueda de dominios incluyen hasta tres valores E diferentes: uno para la secuencia completa de la proteína donde se ha encontrado algún dominio, y dos valores E para el dominio encontrado. Están descritos en las páginas 47 y 48 del documento **HMMER_Userguide.pdf**. ¿Te parecen fiables todos los resultados? ¿Sabrías decir cuántos de los dominios encontrados tiene un valor E condicional (`domain_cevalue`) menor de 1?